### Задание

Реализуйте алгоритм классификации метод k ближайших соседей.

Требования к коду:
* Код должен быть хорошо структурирован
* Код должен быть эффективен
* Имплементация должна быть максимально векторизованной и, где это возможно, не использовать циклы

Необходимо реализовать класс KnnBruteClassifier, с реализацией прототипа, представленного ниже.

Должна быть реализована поддержка метрики расстояния L2 (параметр metric) и параметр weights типа 'uniform' и 'distance'.

В качестве входного файла необходимо использовать файл "knn_data_XXX.npy", полученный от бота командой /get seminar04

В качестве решения необходимо отправить боту, указав seminar04 в поле caption,  следующие файлы:
* knn.ipynb - содержит класс, реализующий ваш алгоритм
* results.npy - файл с результатами тестов, который можно будет сгенерировать с помощью этого ноутбука

Для проверки решения после отправки необходимо отправить боту следующую команду:
/check seminar04

В случае возникновения вопросов по интерфейсу смотрите детали реализации класса sklearn.neighbors.KNeighborsClassifier
https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

https://github.com/esokolov/ml-course-hse/blob/master/2021-spring/seminars/sem19-knn.pdf

In [1]:
import numpy as np
import pandas as pd
import math
import scipy
from sklearn.utils.extmath import weighted_mode
from sklearn.neighbors import KNeighborsClassifier

In [2]:
class KnnBruteClassifier(object):
    '''Классификатор реализует взвешенное голосование по ближайшим соседям. 
    Поиск ближайшего соседа осуществляется полным перебором.
    Параметры
    ----------
    n_neighbors : int, optional
        Число ближайших соседей, учитывающихся в голосовании
    weights : str, optional (default = 'uniform')
        веса, используемые в голосовании. Возможные значения:
        - 'uniform' : все веса равны.
        - 'distance' : веса обратно пропорциональны расстоянию до классифицируемого объекта
        -  функция, которая получает на вход массив расстояний и возвращает массив весов
    metric: функция подсчета расстояния (по умолчанию l2).
    '''
    def __init__(self, n_neighbors=1, weights='uniform', metric="l2"):
        self.n_neighbors = n_neighbors
        self.weights_param = weights
        
        if metric == 'l2':
            self.algo = self.euclidian_distance
        else: 
            raise ValueError(f"Invalid `metric` {metric}")
    
    @staticmethod
    def euclidian_distance(vec, matrix, p=3):
        ''' Вычисляем Евклидово расстояние между вектором vec и каждым вектором из матрицы matrix
        '''
        return np.sqrt(np.sum((matrix - np.tile(vec, (len(matrix), 1))) ** 2, axis=1))

    def _get_weights(self, dist, weights_param):
        ''' Находим веса в зависимости от weights_param:
            - `uniform`: все веса еденичные
            - `distance`: веса обратно проп расстояниям
        '''
        if weights_param == 'uniform':
            return None

        # делаем веса, обраптно проп расстояниям
        # если нашли обхект с 0 дистанцией, то у него вес 1, у остальных - 0
        if weights_param == 'distance':
            with np.errstate(divide='ignore'):
                dist = 1.0 / dist
            inf_mask = np.isinf(dist)
            inf_row = np.any(inf_mask)
            dist[inf_row] = inf_mask[inf_row]
            return dist
    
    def fit(self, x, y):
        ''' Обучение модели.
        Параметры
        ----------
        x : двумерным массив признаков размера n_queries x n_features
        y : массив/список правильных меток размера n_queries
        Выход
        -------
        Метод возвращает обученную модель
        '''
        self.X_train = x
        self.y_train = y
        
    def predict(self, X):
        """ Предсказание класса для входных объектов
        Параметры
        ----------
        X : двумерным массив признаков размера n_queries x n_features
        Выход
        -------
        y : Массив размера n_queries
        """
        # находим ближ соседей и получаем веса соседей по параметру weights_param
        neigh_dist, neigh_indarray = self.kneighbors(X, self.n_neighbors)
        weights = self._get_weights(neigh_dist, self.weights_param)        
        if weights is None:
            mode, _ = scipy.stats.mode(self.y_train[neigh_indarray], axis=1)  
        else:
            mode, _ = weighted_mode(self.y_train[neigh_indarray], weights, axis=1) 
            
        return np.asarray(mode.ravel(), dtype=np.intp)
        
        
    def predict_proba(self, X):
        """Возвращает вероятности классов для входных объектов
        Параметры
        ----------
        X : двумерным массив признаков размера n_queries x n_features
        Выход
        -------
        p : массив размера n_queries x n_classes] c вероятностями принадлежности 
        объекта к каждому классу
        """
        neigh_dist, neigh_indarray = self.kneighbors(X, self.n_neighbors)
        weights = self._get_weights(neigh_dist, self.weights_param) 
        if weights is None:
            weights = np.ones_like(neigh_indarray)

        # итерируясь по каждому виду таргета векторизованно считаем вероятности 
        pred_labels = self.y_train[neigh_indarray]
        probas = []
        for label in set(self.y_train):
            masked = np.ma.masked_array(weights, mask = (pred_labels != label)) # тут векторизованно, но можно и без
            proba_by_label = masked.sum(axis=1).data / weights.sum(axis=1)      # тут векторизованно, но можно и без
            probas.append(proba_by_label)
        
        return np.asarray(list(zip(*probas)), np.float64)
    
    def _get_k_nns(self, x1, k):
        ''' Находим ближайшего соседа из Х_train для х1
        '''
        # считаем расстояния от х1 до каждого объекта в трейне
        dists = self.algo(x1, self.X_train)                                     # тут векторизованно, но можно и без
        sorted_nns = np.array(sorted(zip(range(len(dists)), dists), 
                                     key=lambda x: x[1])[:k])
        dists, idxs = sorted_nns[:,1], sorted_nns[:,0]
        return dists, idxs
        
    def kneighbors(self, x, n_neighbors):
        """ Возвращает n_neighbors ближайших соседей 
            для всех входных объектов и расстояния до них
        Параметры
        ----------
        X : двумерным массив признаков размера n_queries x n_features
        Выход
        -------
        neigh_dist массив размера n_queries х n_neighbors
        расстояния до ближайших элементов
        neigh_indarray, массив размера n_queries x n_neighbors
        индексы ближайших элементов
        """
        # итерируясь по каждому входному объекту находим его nns
        neigh_dist, neigh_indarray = [], []
        for elem in x:
            dists, idxs = self._get_k_nns(elem, k=n_neighbors)
            neigh_dist.append(dists)
            neigh_indarray.append(idxs)

        return np.array(neigh_dist), np.asarray(neigh_indarray, np.int64)

In [3]:
def load_file(filename):
    """
    TODO: Необходимо загрузить файл задания и вернуть словарь с ключами "X_train", "X_test", "y_train"
    """
    return np.load('knn_data_087.npy', allow_pickle=True).item()

In [4]:
input_filename = "knn_data_087.npy" #TODO задать путь к входному файлу
data_dict = load_file(input_filename)

## Пример работы

In [5]:
knnb = KnnBruteClassifier(n_neighbors=1, weights='uniform', metric="l2")
knnb.fit(data_dict["X_train"], data_dict["y_train"])

In [6]:
knnb._get_k_nns(data_dict['X_train'][0], 5)

(array([0.        , 2.02801567, 2.40733786, 2.95657087, 2.95975611]),
 array([  0., 332., 276., 657., 431.]))

In [7]:
a,b = knnb.kneighbors(data_dict['X_test'][30:33], 5)
a,b

(array([[2.69648094, 3.00639148, 3.19284103, 3.27703849, 3.44592179],
        [2.3443735 , 2.48005403, 2.88411034, 3.04010719, 3.10831352],
        [3.00909217, 3.44527439, 4.06177944, 4.11511927, 5.06202939]]),
 array([[345, 285, 102, 461,  45],
        [ 81, 628, 159, 409, 533],
        [305, 234, 260, 613,  40]], dtype=int64))

In [8]:
knnb.predict(data_dict['X_train'][0:5]), data_dict['y_train'][0:5]

(array([8, 5, 5, 6, 5], dtype=int64), array([8, 5, 5, 6, 5], dtype=int64))

In [9]:
knnb.predict(data_dict['X_test'][0:5])

array([2, 6, 7, 8, 1], dtype=int64)

## Прогон модели

In [10]:
model1 = KnnBruteClassifier(n_neighbors=5, weights='uniform')
model1.fit(data_dict["X_train"], data_dict["y_train"])

l2_uniform_n5_y_predict = model1.predict(data_dict["X_test"])
l2_uniform_n5_y_predict_proba = model1.predict_proba(data_dict["X_test"])

In [11]:
model2 = KnnBruteClassifier(n_neighbors=10, weights='uniform')
model2.fit(data_dict["X_train"], data_dict["y_train"])

l2_uniform_10_y_predict = model2.predict(data_dict["X_test"])
l2_uniform_10_y_predict_proba = model2.predict_proba(data_dict["X_test"])

In [12]:
model3 = KnnBruteClassifier(n_neighbors=5, weights='distance')
model3.fit(data_dict["X_train"], data_dict["y_train"])

l2_distance_n5_y_predict = model3.predict(data_dict["X_test"])
l2_distance_n5_y_predict_proba = model3.predict_proba(data_dict["X_test"])

## Проверка на sklearn-модели

In [13]:
check_knn1 = KNeighborsClassifier(n_neighbors=5, weights='uniform', metric='l2', algorithm='brute')
check_knn1.fit(data_dict["X_train"], data_dict["y_train"])

check1 = check_knn1.predict(data_dict["X_test"])
check1_proba = check_knn1.predict_proba(data_dict["X_test"])

In [14]:
check_knn2 = KNeighborsClassifier(n_neighbors=10, weights='uniform', metric='l2', algorithm='brute')
check_knn2.fit(data_dict["X_train"], data_dict["y_train"])

check2 = check_knn2.predict(data_dict["X_test"])
check2_proba = check_knn2.predict_proba(data_dict["X_test"])

In [15]:
check_knn3 = KNeighborsClassifier(n_neighbors=5, weights='distance', metric='l2', algorithm='brute')
check_knn3.fit(data_dict["X_train"], data_dict["y_train"])

check3 = check_knn3.predict(data_dict["X_test"])
check3_proba = check_knn3.predict_proba(data_dict["X_test"])

In [16]:
(check1 != l2_uniform_n5_y_predict).sum(), (check1_proba != l2_uniform_n5_y_predict_proba).sum()

(0, 0)

In [17]:
(check2 != l2_uniform_10_y_predict).sum(), (check2_proba != l2_uniform_10_y_predict_proba).sum()

(0, 0)

In [18]:
(check3 != l2_distance_n5_y_predict).sum(), (check3_proba != l2_distance_n5_y_predict_proba).sum()

(0, 76)

In [19]:
(check3_proba.round(7) != l2_distance_n5_y_predict_proba.round(7)).sum()

0

__То, что у нас не совпала в нек случаях проба В ТОЧНОСТИ - это норм, пример &uarr;__ \
Все из-за гуляющей мантиссы - если округлим до 7го знака, то все норм

## Сохранение модели

In [20]:
# output_filename = "results.npy"
# result_dict = {
#     "input_filename": input_filename,
#     "l2_uniform_n5_y_predict": l2_uniform_n5_y_predict,
#     "l2_uniform_10_y_predict": l2_uniform_10_y_predict,
#     "l2_distance_n5_y_predict": l2_distance_n5_y_predict,
# }
# np.save(output_filename, result_dict)